# Information Retrieval Lab WiSe 2024/2025: Baseline Retrieval System

This Jupyter notebook serves as a baseline retrieval system that you can improve upon.
We use subsets of the MS MARCO datasets to retrieve passages of web documents.
We will show you how to create a software submission to TIRA from this notebook.

An overview of all corpora that we use in the current course is available at [https://tira.io/datasets?query=ir-lab-wise-2024](https://tira.io/datasets?query=ir-lab-wise-2024). The dataset IDs for loading the datasets are:

- `ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training`: A subsample of the TREC 2019/2020 Deep Learning tracks on the MS MARCO v1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/subsampled-ms-marco-rag-20241202-training` (_work in progress_): A subsample of the TREC 2024 Retrieval-Augmented Generation track on the MS MARCO v2.1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/ms-marco-rag-20241203-test` (work in progress): The test corpus that we have created together in the course, based on the MS MARCO v2.1 passage dataset. We will use this dataset as the test dataset, i.e., evaluation scores become available only after the submission deadline.

### Step 1: Import libraries

We will use [tira](https://tira.io/), an information retrieval shared task platform, and [ir_dataset](https://ir-datasets.com/) for loading the datasets. Subsequently, we will build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine framework.

First, we need to install the required libraries.

In [1]:
!pip3 install 'tira>=0.0.139' ir-datasets 'python-terrier==0.10.0'

In [2]:
!wget https://files.webis.de/software/pyterrier-plugins/custom-terrier-token-processing-1.0-SNAPSHOT-jar-with-dependencies.jar -O /root/.pyterrier/custom-terrier-token-processing-0.0.1.jar

--2024-12-09 10:04:46--  https://files.webis.de/software/pyterrier-plugins/custom-terrier-token-processing-1.0-SNAPSHOT-jar-with-dependencies.jar
Resolving files.webis.de (files.webis.de)... 141.54.132.200
Connecting to files.webis.de (files.webis.de)|141.54.132.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 499865236 (477M) [application/java-archive]
Saving to: ‘/root/.pyterrier/custom-terrier-token-processing-0.0.1.jar’

/root/.pyterrier/cu 100%[===================>] 476.71M   107MB/s    in 4.5s    

2024-12-09 10:04:51 (106 MB/s) - ‘/root/.pyterrier/custom-terrier-token-processing-0.0.1.jar’ saved [499865236/499865236]



In [3]:
import pyterrier as pt
import pandas as pd
pd.set_option('display.max_colwidth', 0)

if not pt.started():
    pt.init(boot_packages=['mam10eks:custom-terrier-token-processing:0.0.1'])
    from jnius import autoclass

PyTerrier 0.10.0 has loaded Terrier 5.10 (built by craigm on 2024-08-22 17:33) and terrier-helper 0.0.8



Create an API client to interact with the TIRA platform (e.g., to load datasets and submit runs).

In [4]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client

ensure_pyterrier_is_loaded()
tira = Client()

### Step 2: Load the dataset

We load the dataset by its ir_datasets ID (as listed in the Readme). Just be sure to add the `irds:` prefix before the dataset ID to tell PyTerrier to load the data from ir_datasets.

In [5]:
from pyterrier import get_dataset

pt_dataset = get_dataset('irds:ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training')

In [6]:
from tira.third_party_integrations import ir_datasets, ensure_pyterrier_is_loaded, persist_and_normalize_run
import pyterrier as pt

ensure_pyterrier_is_loaded()

training_dataset = 'ir-lab-jena-leipzig-wise-2023/training-20231104-training'
validation_dataset = 'ir-lab-jena-leipzig-wise-2023/validation-20231104-training'

In [7]:
pt_queries = pt.io.read_topics(ir_datasets.topics_file(training_dataset), format='trecxml')

### Step 3: Build an index

We will then create an index from the documents in the dataset we just loaded.

In [8]:
from pyterrier import IterDictIndexer

indexer_porter = IterDictIndexer(
    # Store the index in the `index` directory.
    "../data/index",
    meta={'docno': 50, 'text': 4096},
    # If an index already exists there, then overwrite it.
    overwrite=True,
    stemmer='PorterStemmer'
)
index_porter = indexer_porter.index(pt_dataset.get_corpus_iter())


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  38%|███▊      | 26075/68261 [00:07<00:08, 4820.75it/s]

10:05:19.487 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [00:13<00:00, 5141.23it/s]


10:05:27.222 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


In [9]:
indexer_none = IterDictIndexer(
    "../data/index_none",
    meta={'docno': 50, 'text': 4096},
    overwrite=True,
    stemmer = None
)

index_none= indexer_none.index(pt_dataset.get_corpus_iter())

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  38%|███▊      | 25912/68261 [00:03<00:04, 9319.84it/s] 

10:05:32.610 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [00:08<00:00, 7887.29it/s] 


10:05:39.636 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


In [10]:
indexer_Snowball = IterDictIndexer(
    "../data/index_Snowball",
    meta={'docno': 50, 'text': 4096},
    overwrite=True,
    stemmer = 'EnglishSnowballStemmer'
)

index_Snowball = indexer_Snowball.index(pt_dataset.get_corpus_iter())

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  37%|███▋      | 25488/68261 [00:03<00:05, 7339.34it/s]

10:05:45.803 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [00:08<00:00, 7595.56it/s]


10:05:52.623 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


In [11]:
indexer_LemurKrovetz = IterDictIndexer(
    "../data/index_LemurKrovetz",
    meta={'docno': 50, 'text': 4096},
    overwrite=True,
    stemmer = 'LemurKrovetzStemmer'
)

index_LemurKrovetz = indexer_LemurKrovetz.index(pt_dataset.get_corpus_iter())

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  38%|███▊      | 25865/68261 [00:05<00:14, 3021.41it/s]

10:06:00.860 [ForkJoinPool-4-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [00:10<00:00, 6293.31it/s] 


10:06:07.930 [ForkJoinPool-4-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


In [12]:
indexer_standfordLemmatizer = IterDictIndexer(
    "../data/index_standfordLemmatizer",
    meta={'docno': 50, 'text': 4096},
    overwrite=True,
    stemmer = 'StanfordLemmatizer'
)

index_standfordLemmatizer = indexer_standfordLemmatizer.index(pt_dataset.get_corpus_iter())

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  38%|███▊      | 26169/68261 [00:37<00:52, 802.15it/s]

10:06:49.376 [ForkJoinPool-5-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [01:32<00:00, 737.46it/s]


10:07:46.082 [ForkJoinPool-5-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


### Step 4: Define the retrieval pipeline

We will define a simple retrieval pipeline using just BM25 as a baseline. For details, refer to the PyTerrier [documentation](https://pyterrier.readthedocs.io) or [tutorial](https://github.com/terrier-org/ecir2021tutorial).

In [13]:
from pyterrier import BatchRetrieve

bm25_porter = BatchRetrieve(index_porter, wmodel="BM25")
bm25_none = BatchRetrieve(index_none, wmodel="BM25")
bm25_snowball = BatchRetrieve(indexer_Snowball, wmodel="BM25")
bm25_lemurkrovetz = BatchRetrieve(indexer_LemurKrovetz, wmodel="BM25")
bm25_standfordLemmatizer = BatchRetrieve(index_standfordLemmatizer, wmodel="BM25")

In [14]:
from tira.third_party_integrations import ir_datasets, ensure_pyterrier_is_loaded, persist_and_normalize_run
def run_bm25_grid_search_run(index, output_dir, queries):
    """
        defaults: http://terrier.org/docs/current/javadoc/org/terrier/matching/models/BM25.html
        k_1 = 1.2d, k_3 = 8d, b = 0.75d
        We do not tune parameter k_3, as this parameter only impacts queries with reduntant terms.
    """
    for b in [0.7, 0.75, 0.8]:
        for k_1 in [1.1, 1.2, 1.3]:
            system = f'bm25-b={b}-k_1={k_1}'
            configuration = {"bm25.b" : b, "bm25.k_1": k_1}
            run_output_dir = output_dir + '/' + system
            !rm -Rf {run_output_dir}
            !mkdir -p {run_output_dir}
            print(f'Run {system}')
            BM25 = pt.BatchRetrieve(index, wmodel="BM25", controls=configuration, verbose=True)
            run = BM25(queries)
            persist_and_normalize_run(run, system, run_output_dir)

### Load Training Dataset

In [15]:
run_bm25_grid_search_run(index_LemurKrovetz, 'grid-search/training', pt_queries)

Run bm25-b=0.7-k_1=1.1


BR(BM25): 100%|██████████| 672/672 [00:29<00:00, 22.50q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/training/bm25-b=0.7-k_1=1.1".
Done. run file is stored under "grid-search/training/bm25-b=0.7-k_1=1.1/run.txt".
Run bm25-b=0.7-k_1=1.2


BR(BM25): 100%|██████████| 672/672 [00:28<00:00, 23.25q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/training/bm25-b=0.7-k_1=1.2".
Done. run file is stored under "grid-search/training/bm25-b=0.7-k_1=1.2/run.txt".
Run bm25-b=0.7-k_1=1.3


...

The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/training/bm25-b=0.8-k_1=1.1".
Done. run file is stored under "grid-search/training/bm25-b=0.8-k_1=1.1/run.txt".
Run bm25-b=0.8-k_1=1.2


BR(BM25): 100%|██████████| 672/672 [00:34<00:00, 19.61q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/training/bm25-b=0.8-k_1=1.2".
Done. run file is stored under "grid-search/training/bm25-b=0.8-k_1=1.2/run.txt".
Run bm25-b=0.8-k_1=1.3


BR(BM25): 100%|██████████| 672/672 [00:33<00:00, 20.00q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/training/bm25-b=0.8-k_1=1.3".
Done. run file is stored under "grid-search/training/bm25-b=0.8-k_1=1.3/run.txt".


### Load Validation Dataset

In [16]:
dataset_val = ir_datasets.load(validation_dataset)
queries_val = pt.io.read_topics(ir_datasets.topics_file(validation_dataset), format='trecxml')

The download is derived from The LongEval Dataset under the "Qwant LongEval Attribution-NonCommercial-ShareAlike License". Hence, the download is also under this License. By using it, you agree to the terms of this license. Please find details at: https://lindat.mff.cuni.cz/repository/xmlui/page/Qwant_LongEval_BY-NC-SA_License
Download from Zenodo: https://zenodo.org/records/10628882/files/validation-20231104-truth.zip?download=1


Download: 100%|██████████| 479k/479k [00:00<00:00, 4.39MiB/s]

Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-jena-leipzig-wise-2023/validation-20231104-training/


In [17]:
run_bm25_grid_search_run(index_LemurKrovetz, 'grid-search/validation', queries_val)

Run bm25-b=0.7-k_1=1.1


BR(BM25): 100%|██████████| 882/882 [00:43<00:00, 20.17q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/validation/bm25-b=0.7-k_1=1.1".
Done. run file is stored under "grid-search/validation/bm25-b=0.7-k_1=1.1/run.txt".
Run bm25-b=0.7-k_1=1.2


BR(BM25): 100%|██████████| 882/882 [00:45<00:00, 19.49q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/validation/bm25-b=0.7-k_1=1.2".
Done. run file is stored under "grid-search/validation/bm25-b=0.7-k_1=1.2/run.txt".
Run bm25-b=0.7-k_1=1.3


...

The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/validation/bm25-b=0.8-k_1=1.1".
Done. run file is stored under "grid-search/validation/bm25-b=0.8-k_1=1.1/run.txt".
Run bm25-b=0.8-k_1=1.2


BR(BM25): 100%|██████████| 882/882 [00:48<00:00, 18.34q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/validation/bm25-b=0.8-k_1=1.2".
Done. run file is stored under "grid-search/validation/bm25-b=0.8-k_1=1.2/run.txt".
Run bm25-b=0.8-k_1=1.3


BR(BM25): 100%|██████████| 882/882 [00:52<00:00, 16.82q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/validation/bm25-b=0.8-k_1=1.3".
Done. run file is stored under "grid-search/validation/bm25-b=0.8-k_1=1.3/run.txt".


### Evalutation

In [19]:
!pip3 install tira trectools python-terrier

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 70.0 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 75.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 KB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 85.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 KB 54.8 MB/s eta 0:00:00
  Created wheel for trectools: filename=trectools-0.0.50-py3-none-any.whl size=28599 sha256=b6587bfcdaa1f4e5b1aff5b552a5d6aca21ba505de8b078f6c5a474c1a0a3951
  Stored in directory: /root/.cache/pip/wheels/48/f7/1d/79a8e3f8fa6e794cd08f9a14df11eb6ce933416febcdd1e612
Successfully built trectools


In [20]:
from trectools import TrecRun, TrecQrel, TrecEval
from tira.rest_api_client import Client
from glob import glob
import pandas as pd
tira = Client()

def load_qrels(dataset):
    return TrecQrel(tira.download_dataset('ir-lab-jena-leipzig-wise-2023', dataset, truth_dataset=True) + '/qrels.txt')

training_qrels = load_qrels('training-20231104-training')
validation_qrels = load_qrels('validation-20231104-training')

In [21]:
def evaluate_run(run_dir, qrels):
    run = TrecRun(run_dir + '/run.txt')
    trec_eval = TrecEval(run, qrels)

    return {
        'run': run.get_runid(),
        'nDCG@10': trec_eval.get_ndcg(depth=10),
        'nDCG@10 (unjudgedRemoved)': trec_eval.get_ndcg(depth=10, removeUnjudged=True),
        'MAP': trec_eval.get_map(depth=10),
        'MRR': trec_eval.get_reciprocal_rank()
    }

In [22]:
df = []
for r in glob('grid-search/training/bm25*'):
    df += [evaluate_run(r, training_qrels)]
df = pd.DataFrame(df)
df.sort_values('nDCG@10', ascending=False)

/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:311: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()
/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:311: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()
/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:311: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current 

,run,nDCG@10,nDCG@10 (unjudgedRemoved),MAP,MRR
0,bm25-b=0.75-k_1=1.3,0.0,0.0,0.0,0.0
1,bm25-b=0.75-k_1=1.2,0.0,0.0,0.0,0.0
2,bm25-b=0.8-k_1=1.1,0.0,0.0,0.0,0.0
3,bm25-b=0.7-k_1=1.3,0.0,0.0,0.0,0.0
4,bm25-b=0.8-k_1=1.2,0.0,0.0,0.0,0.0
5,bm25-b=0.7-k_1=1.2,0.0,0.0,0.0,0.0
6,bm25-b=0.8-k_1=1.3,0.0,0.0,0.0,0.0
7,bm25-b=0.75-k_1=1.1,0.0,0.0,0.0,0.0
8,bm25-b=0.7-k_1=1.1,0.0,0.0,0.0,0.0


In [23]:
df = []
for r in glob('grid-search/validation/bm25*'):
    df += [evaluate_run(r, validation_qrels)]
df = pd.DataFrame(df)
df.sort_values('nDCG@10', ascending=False)

/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:311: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()
/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:311: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()
/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:311: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current 

,run,nDCG@10,nDCG@10 (unjudgedRemoved),MAP,MRR
0,bm25-b=0.75-k_1=1.3,0.0,0.0,0.0,0.0
1,bm25-b=0.75-k_1=1.2,0.0,0.0,0.0,0.0
2,bm25-b=0.8-k_1=1.1,0.0,0.0,0.0,0.0
3,bm25-b=0.7-k_1=1.3,0.0,0.0,0.0,0.0
4,bm25-b=0.8-k_1=1.2,0.0,0.0,0.0,0.0
5,bm25-b=0.7-k_1=1.2,0.0,0.0,0.0,0.0
6,bm25-b=0.8-k_1=1.3,0.0,0.0,0.0,0.0
7,bm25-b=0.75-k_1=1.1,0.0,0.0,0.0,0.0
8,bm25-b=0.7-k_1=1.1,0.0,0.0,0.0,0.0


### Step 5: Create the run
In the next steps, we would like to apply our retrieval system to some topics, to prepare a 'run' file, containing the retrieved documents.

First, let's have a short look at the first three topics:

In [12]:
# The `'text'` argument below selects the topics `text` field as the query.
pt_dataset.get_topics('text').head(3)

,qid,query
0,1030303,who is aziz hashim
1,1037496,who is rep scalise
2,1043135,who killed nicholas ii of russia


Now, retrieve results for all the topics (may take a while):

In [14]:
run = bm25_lemurkrovetz(pt_dataset.get_topics('text'))

That's it for the retrieval. Here are the first 10 entries of the run:

In [15]:
run.head(10)

,qid,docid,docno,rank,score,query
0,1030303,53852,8726436,0,31.681671,who is aziz hashim
1,1030303,56041,8726433,1,25.966276,who is aziz hashim
2,1030303,62116,8726435,2,23.863442,who is aziz hashim
3,1030303,32183,8726429,3,23.391821,who is aziz hashim
4,1030303,35867,8726437,4,21.030669,who is aziz hashim
5,1030303,17637,8726430,5,19.967200,who is aziz hashim
6,1030303,42957,7156982,6,19.967200,who is aziz hashim
7,1030303,21803,8726434,7,19.474804,who is aziz hashim
8,1030303,59828,1305520,8,17.849161,who is aziz hashim
9,1030303,60002,3302257,9,17.832781,who is aziz hashim


### Step 6: Persist and upload run to TIRA

The output of our retrieval system is a run file. This run file can later (and, e.g., in a different notebook or by a different person) be statistically evaluated. We will therefore first upload the run to TIRA.

In [16]:
from tira.third_party_integrations import persist_and_normalize_run

persist_and_normalize_run(
    run,
    # Give your approach a short but descriptive name tag.
    system_name='bm25-modifiedStemmer', 
    default_output='../data/runs',
    upload_to_tira=pt_dataset,
)

The run file is normalized outside the TIRA sandbox, I will store it at "../data/runs".
Done. run file is stored under "../data/runs/run.txt.gz".
Run uploaded to TIRA. Claim ownership via: https://www.tira.io/claim-submission/f3476d3b-6066-434b-8df1-1bc77b9fdfd7


Click on the link in the cell output above to claim your submission on TIRA.

# Step 7: Improve

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [13]:
from pyterrier import Experiment 

Experiment(
    [bm25_porter, bm25_none, bm25_snowball, bm25_lemurkrovetz, bm25_standfordLemmatizer],
    topics = pt_dataset.get_topics("text"),
    qrels = pt_dataset.get_qrels(),
    eval_metrics=["ndcg_cut_10"],
    names=["BM25 with Porter Stemmer", "BM25 with no Stemmer", "BM25 with Snowball Stemmer", "BM25 with LemurKrovetz Stemmer", "BM25 with Standford Lemmatizer"]
)

,name,ndcg_cut_10
0,BM25 with Porter Stemmer,0.489469
1,BM25 with no Stemmer,0.468890
2,BM25 with Snowball Stemmer,0.489216
3,BM25 with LemurKrovetz Stemmer,0.490341
4,BM25 with Standford Lemmatizer,0.482512


In [10]:
pt_dataset.get_qrels()

,qid,docno,label,iteration
0,19335,1082489,0,0
1,19335,109063,0,0
2,19335,1231806,0,0
3,19335,1324075,0,0
4,19335,1509459,0,0
...,...,...,...,...
12971,1136962,8336581,0,0
12972,1136962,8380913,0,0
12973,1136962,8537921,0,0
12974,1136962,937258,1,0
